# Analytics

In [7]:
import pandas as pd
import yfinance as yf

PERIODS = ["1D", "1W", "1M", "YTD", "1Y", "3Y", "5Y"]

def _to_ts(d):
    if d is None:
        return None
    return pd.Timestamp(d).normalize()

def _nearest_prev_close(close: pd.Series, target_date: pd.Timestamp):
    s = close.loc[:target_date]
    if s.empty:
        return None, None
    return float(s.iloc[-1]), s.index[-1]

def yahoo_perf_asof(ticker: str, asof=None, auto_adjust=True) -> pd.DataFrame:
    """
    asof: str|date|datetime|Timestamp|None
      - None => prend le dernier close dispo
      - sinon => prend le dernier close <= asof
    """
    t = yf.Ticker(ticker)

    # On prend large pour couvrir 5Y même si asof est dans le passé
    # (si asof est très vieux, augmente period ou utilise start/end)
    hist = t.history(period="10y", interval="1d", auto_adjust=auto_adjust)
    if hist.empty:
        raise ValueError(f"No data for ticker '{ticker}'")

    close = hist["Close"].dropna()
    close.index = pd.to_datetime(close.index).tz_localize(None)

    asof_ts = _to_ts(asof)

    # Last price à asOf (dernier close <= asof)
    if asof_ts is None:
        last_price = float(close.iloc[-1])
        last_date = close.index[-1]
    else:
        last_price, last_date = _nearest_prev_close(close, asof_ts)
        if last_price is None:
            raise ValueError(f"No data on or before asof={asof_ts.date()} for '{ticker}'")

    # Dates cibles relatives à last_date (qui est le trading day retenu)
    targets = {
        "1D": last_date - pd.Timedelta(days=1),
        "1W": last_date - pd.Timedelta(days=7),
        "1M": last_date - pd.Timedelta(days=30),
        "YTD": pd.Timestamp(year=last_date.year, month=1, day=1),
        "1Y": last_date - pd.Timedelta(days=365),
        "3Y": last_date - pd.Timedelta(days=365 * 3),
        "5Y": last_date - pd.Timedelta(days=365 * 5),
    }

    out = {
        "Ticker": ticker,
        "AsOfRequested": None if asof_ts is None else asof_ts.date(),
        "AsOfUsed": last_date.date(),        # trading day effectivement utilisé
        "Last": last_price,
    }

    for k, d in targets.items():
        past_price, past_date = _nearest_prev_close(close, d)
        if past_price is None or past_price == 0:
            out[k] = None
        else:
            out[k] = (last_price / past_price - 1) * 100.0

    return pd.DataFrame([out])

def yahoo_perf_table_asof(tickers, asof=None, auto_adjust=True) -> pd.DataFrame:
    dfs = [yahoo_perf_asof(t, asof=asof, auto_adjust=auto_adjust) for t in tickers]
    df = pd.concat(dfs, ignore_index=True)

    pct_cols = ["1D", "1W", "1M", "YTD", "1Y", "3Y", "5Y"]
    df["Last"] = df["Last"].round(6)
    df[pct_cols] = df[pct_cols].round(2)
    return df


In [10]:
df = yahoo_perf_table_asof(
    ["AAPL", "SPY", "AIR.PA", "IWDA.AS"],
    asof="2025-11-01"  # <= tu changes ça quand tu veux
)
pct_cols = ["1D", "1W", "1M", "YTD", "1Y", "3Y", "5Y"]
df.style.format({**{"Last":"{:.2f}"}, **{c:"{:+.2f}%" for c in pct_cols}})

,Ticker,AsOfRequested,AsOfUsed,Last,1D,1W,1M,YTD,1Y,3Y,5Y
0,AAPL,2025-11-01,2025-10-31,270.11,-0.38%,+2.87%,+5.84%,+8.35%,+20.24%,+82.30%,+155.33%
1,SPY,2025-11-01,2025-10-31,680.05,+0.33%,+0.71%,+2.04%,+17.40%,+21.40%,+84.83%,+124.05%
2,AIR.PA,2025-11-01,2025-10-31,213.40,+0.33%,+2.37%,+7.19%,+40.97%,+55.68%,+100.10%,+264.89%
3,IWDA.AS,2025-11-01,2025-10-31,111.79,-0.12%,+1.09%,+3.49%,+7.32%,+14.59%,+54.26%,+108.89%
